In [27]:
import xarray as xr
from datetime import datetime as dt
import numpy as np
from os import listdir, path

In [3]:
date_i_want = dt(2022, 6, 2)

tfm = xr.open_dataset(f'/Volumes/LtgSSD/tobac_saves/tobac_Save_{date_i_want.strftime('%Y%m%d')}/Track_features_merges.nc', chunks='auto')

In [51]:
radar_top_path = f'/Volumes/LtgSSD/nexrad_zarr/{date_i_want.strftime('%B').upper()}/{date_i_want.strftime('%Y%m%d')}/'
radar_files = listdir(radar_top_path)
radar_dts = [dt.strptime(rf, 'KHGX%Y%m%d_%H%M%S_V06_grid.zarr') for rf in radar_files if rf.endswith('.zarr')]
radar_dts = np.array(radar_dts).astype('datetime64[s]').astype(dt)
radar_files = [path.join(radar_top_path, rf) for rf in radar_files if rf.endswith('.zarr')]

In [ ]:
all_feature_eet = xr.zeros_like(tfm.feature)
for i, feature_id in enumerate(tfm.feature.data):
    print(feature_id)
    this_feature = tfm.sel(feature=feature_id)
    this_feature_dt = this_feature.feature_time.data.compute().astype('datetime64[s]').item()
    radar_path_i_want = radar_files[np.argmin(np.abs(radar_dts - this_feature_dt))]
    radar = xr.open_dataset(radar_path_i_want, engine='zarr', chunks='auto')
    this_eet = radar.eet_sam.isel(time=0)
    this_seg_mask = this_feature.sel(time=this_feature_dt, method='nearest').segmentation_mask
    feature_eet = np.nanmax(np.where(this_seg_mask.data == feature_id, this_eet, 0)).compute()
    all_feature_eet[i] = feature_eet


1
